In [6]:
# !kaggle datasets download -d trolukovich/apparel-images-dataset
# !kaggle datasets download -d airplane2230/apparel-image-dataset-2

Dataset URL: https://www.kaggle.com/datasets/airplane2230/apparel-image-dataset-2


  0%|          | 0.00/249M [00:00<?, ?B/s]
  0%|          | 1.00M/249M [00:00<02:37, 1.65MB/s]
  1%|          | 2.00M/249M [00:00<01:24, 3.06MB/s]
  1%|          | 3.00M/249M [00:00<00:57, 4.45MB/s]
  2%|▏         | 4.00M/249M [00:01<00:50, 5.10MB/s]
  2%|▏         | 6.00M/249M [00:01<00:34, 7.32MB/s]
  3%|▎         | 7.00M/249M [00:01<00:32, 7.78MB/s]
  3%|▎         | 8.00M/249M [00:01<00:30, 8.16MB/s]
  4%|▎         | 9.00M/249M [00:01<00:32, 7.73MB/s]
  4%|▍         | 11.0M/249M [00:01<00:28, 8.77MB/s]
  5%|▌         | 13.0M/249M [00:01<00:26, 9.31MB/s]
  6%|▌         | 14.0M/249M [00:02<00:27, 9.11MB/s]
  6%|▋         | 16.0M/249M [00:02<00:26, 9.13MB/s]
  7%|▋         | 17.0M/249M [00:02<00:26, 9.25MB/s]
  8%|▊         | 19.0M/249M [00:02<00:25, 9.52MB/s]
  8%|▊         | 20.0M/249M [00:02<00:27, 8.65MB/s]
  9%|▉         | 22.0M/249M [00:03<00:25, 9.20MB/s]
 10%|▉         | 24.0M/249M [00:03<00:24, 9.44MB/s]
 10%|█         | 25.0M/249M [00:03<00:24, 9.55MB/s]
 11%|█         | 27.


License(s): CC0-1.0



In [7]:
## 압축 해제
# import shutil
# shutil.unpack_archive("apparel-image-dataset-2.zip", "./")

In [8]:
import pandas as pd
train_df = pd.read_csv("train.csv", index_col=0)
train_df.head()

,image,black,blue,brown,green,red,white,dress,shirt,pants,shorts,shoes
0,./clothes_dataset\blue_shorts\256d854b55ac32ea...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,./clothes_dataset\red_pants\584f778aece14f07c2...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,./clothes_dataset\green_pants\ec543ca241cefb2b...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,./clothes_dataset\brown_shorts\c8db9e0f7010592...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,./clothes_dataset\white_dress\551373c80717c5b0...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [19]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
import numpy as np

# 멀티레이블을 위한 커스텀 데이터셋 클래스
class ClothesDataset(Dataset):
    def __init__(self, df, transform=None):
        self.transform = transform
        self.image_paths = np.array(df['image'])
        self.labels = np.array(df.iloc[:, 1:])

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

In [20]:
# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [16]:
test_df = pd.read_csv("test.csv", index_col=0)
test_df.head()

,image,black,blue,brown,green,red,white,dress,shirt,pants,shorts,shoes
0,./clothes_dataset\blue_shorts\1ec871cdfe41ff20...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,./clothes_dataset\black_pants\0f8eb4509cea01f9...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,./clothes_dataset\white_shoes\1d70ebaec12e09f3...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,./clothes_dataset\green_shoes\a4e155f7249a591e...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,./clothes_dataset\blue_shirt\2edd17185a4f1b8d9...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [21]:
batch_size = 32
# 데이터 셋 및 데이터 로더 생성
dataset = ClothesDataset(df=train_df, transform=transform)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

test_dataset = ClothesDataset(df=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [22]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
len(train_df.columns) - 1

11

In [24]:
# 전이학습
import torch.nn as nn

for name, module in model.named_parameters():
    module.requires_grad = False

In [25]:
for name, module in model.named_parameters():
    print(module.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [26]:
model.fc = nn.Linear(512, 11)

In [27]:
device = torch.device("cuda")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

for epochs in range(5):
    loss_sum = 0
    cnt = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        cnt += 1

        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)

        loss = criterion(outputs, labels)
        loss_sum += loss.item()
        loss.backward()
        optimizer.step()
    print(f"epoch : {epochs + 1}, loss : {loss_sum/cnt}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_17084\9732327.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


epoch : 1, loss : 0.2638653062922614
epoch : 2, loss : 0.14079637974500656
epoch : 3, loss : 0.11270187948431287
epoch : 4, loss : 0.10194286833916391
epoch : 5, loss : 0.09144565514155796


In [29]:
model.eval()
correct = 0
cnt = 0
with torch.no_grad():
    for img,label in test_loader:
        outputs = model(img.to('cuda'))
        greater_than_half = nn.Sigmoid()(model(img.to('cuda'))) > 0.5
        result = torch.nonzero(greater_than_half)

        label_result = label > 0.5
        result1 = torch.nonzero(label_result)

        for temp,temp1 in zip(result,result1):
            result_data = torch.equal(temp.to('cpu'),temp1.to('cpu'))
            if result_data:
                correct += 1
            cnt += 1


    accuracy = correct / cnt
    print(f'accuracy : {accuracy}')

C:\Users\Admin\AppData\Local\Temp\ipykernel_17084\9732327.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


accuracy : 0.205545229244114


In [30]:
greater_than_half = nn.Sigmoid()(model(img.to('cuda'))) > 0.5
result = torch.nonzero(greater_than_half)
result

tensor([[ 0,  1],
        [ 0,  8],
        [ 1, 10],
        [ 2,  1],
        [ 2,  6],
        [ 3,  1],
        [ 3, 10],
        [ 4,  3],
        [ 4, 10],
        [ 5,  1],
        [ 5,  8],
        [ 6,  3],
        [ 6,  8],
        [ 7,  8],
        [ 8,  1],
        [ 8,  8],
        [ 9,  2],
        [ 9, 10],
        [10,  5],
        [10,  8],
        [11,  4],
        [11,  8],
        [12,  3],
        [12,  7],
        [13,  0],
        [13, 10],
        [14,  0],
        [15,  2],
        [15, 10],
        [16,  5],
        [16,  6],
        [17,  0],
        [18,  1],
        [18,  7],
        [19,  4],
        [19,  8],
        [20,  1],
        [20,  8],
        [21,  8],
        [22,  1],
        [22,  8],
        [23,  4],
        [23,  6]], device='cuda:0')

In [31]:
label_result = label > 0.5
result1 = torch.nonzero(greater_than_half)
result1

tensor([[ 0,  1],
        [ 0,  8],
        [ 1, 10],
        [ 2,  1],
        [ 2,  6],
        [ 3,  1],
        [ 3, 10],
        [ 4,  3],
        [ 4, 10],
        [ 5,  1],
        [ 5,  8],
        [ 6,  3],
        [ 6,  8],
        [ 7,  8],
        [ 8,  1],
        [ 8,  8],
        [ 9,  2],
        [ 9, 10],
        [10,  5],
        [10,  8],
        [11,  4],
        [11,  8],
        [12,  3],
        [12,  7],
        [13,  0],
        [13, 10],
        [14,  0],
        [15,  2],
        [15, 10],
        [16,  5],
        [16,  6],
        [17,  0],
        [18,  1],
        [18,  7],
        [19,  4],
        [19,  8],
        [20,  1],
        [20,  8],
        [21,  8],
        [22,  1],
        [22,  8],
        [23,  4],
        [23,  6]], device='cuda:0')

In [32]:
true_sum = 0

for temp,temp1 in zip(result,result1):
    print(temp.to('cpu'),temp1.to('cpu'))
    result_data = torch.equal(temp.to('cpu'),temp1.to('cpu'))
    if result_data:
        true_sum += 1
print(true_sum/len(result))

tensor([0, 1]) tensor([0, 1])
tensor([0, 8]) tensor([0, 8])
tensor([ 1, 10]) tensor([ 1, 10])
tensor([2, 1]) tensor([2, 1])
tensor([2, 6]) tensor([2, 6])
tensor([3, 1]) tensor([3, 1])
tensor([ 3, 10]) tensor([ 3, 10])
tensor([4, 3]) tensor([4, 3])
tensor([ 4, 10]) tensor([ 4, 10])
tensor([5, 1]) tensor([5, 1])
tensor([5, 8]) tensor([5, 8])
tensor([6, 3]) tensor([6, 3])
tensor([6, 8]) tensor([6, 8])
tensor([7, 8]) tensor([7, 8])
tensor([8, 1]) tensor([8, 1])
tensor([8, 8]) tensor([8, 8])
tensor([9, 2]) tensor([9, 2])
tensor([ 9, 10]) tensor([ 9, 10])
tensor([10,  5]) tensor([10,  5])
tensor([10,  8]) tensor([10,  8])
tensor([11,  4]) tensor([11,  4])
tensor([11,  8]) tensor([11,  8])
tensor([12,  3]) tensor([12,  3])
tensor([12,  7]) tensor([12,  7])
tensor([13,  0]) tensor([13,  0])
tensor([13, 10]) tensor([13, 10])
tensor([14,  0]) tensor([14,  0])
tensor([15,  2]) tensor([15,  2])
tensor([15, 10]) tensor([15, 10])
tensor([16,  5]) tensor([16,  5])
tensor([16,  6]) tensor([16,  6])
te

In [33]:
model.eval()
nn.Sigmoid()(model(img.to('cuda')))
greater_than_half = nn.Sigmoid()(model(img.to('cuda'))) > 0.5
result = torch.nonzero(greater_than_half)
result

tensor([[ 0,  1],
        [ 0,  8],
        [ 1, 10],
        [ 2,  1],
        [ 2,  6],
        [ 3,  1],
        [ 3, 10],
        [ 4,  3],
        [ 4, 10],
        [ 5,  1],
        [ 5,  8],
        [ 6,  3],
        [ 6,  8],
        [ 7,  8],
        [ 8,  1],
        [ 8,  8],
        [ 9,  2],
        [ 9, 10],
        [10,  5],
        [10,  8],
        [11,  4],
        [11,  8],
        [12,  3],
        [12,  7],
        [13,  0],
        [13, 10],
        [14,  0],
        [15,  2],
        [15, 10],
        [16,  5],
        [16,  6],
        [17,  0],
        [18,  1],
        [18,  7],
        [19,  4],
        [19,  8],
        [20,  1],
        [20,  8],
        [21,  8],
        [22,  1],
        [22,  8],
        [23,  4],
        [23,  6]], device='cuda:0')

In [34]:
train_df.columns[[0,4]]

Index(['image', 'green'], dtype='object')

In [35]:
greater_than_half = label > 0.5
result = torch.nonzero(greater_than_half)
result

tensor([[ 0,  1],
        [ 0,  8],
        [ 1,  0],
        [ 1, 10],
        [ 2,  1],
        [ 2,  6],
        [ 3,  1],
        [ 3, 10],
        [ 4,  3],
        [ 4, 10],
        [ 5,  1],
        [ 5,  8],
        [ 6,  3],
        [ 6,  8],
        [ 7,  0],
        [ 7,  8],
        [ 8,  1],
        [ 8,  8],
        [ 9,  2],
        [ 9, 10],
        [10,  5],
        [10,  8],
        [11,  4],
        [11,  8],
        [12,  3],
        [12,  7],
        [13,  0],
        [13, 10],
        [14,  0],
        [14,  7],
        [15,  2],
        [15, 10],
        [16,  5],
        [16,  6],
        [17,  0],
        [17,  6],
        [18,  1],
        [18,  7],
        [19,  4],
        [19,  8],
        [20,  1],
        [20,  8],
        [21,  0],
        [21,  8],
        [22,  1],
        [22,  8],
        [23,  4],
        [23,  6]])

In [36]:
# 예시 데이터프레임 생성
data = {
    'image': ['path_to_image1.jpg', 'path_to_image2.jpg', 'path_to_image3.jpg', 'path_to_image4.jpg'],
    'black': [1, 0, 0, 1],
    'blue': [0, 1, 1, 0],
    'brown': [0, 1, 0, 0],
    'green': [0, 0, 1, 0],
    'red': [1, 0, 0, 1],
    'white': [0, 1, 0, 0],
    'dress': [1, 0, 0, 1],
    'pants': [0, 1, 1, 0],
    'shirt': [0, 0, 1, 0],
    'shoes': [0, 0, 0, 1],
    'short': [0, 0, 0, 0]
}

df = pd.DataFrame(data)
df

,image,black,blue,brown,green,red,white,dress,pants,shirt,shoes,short
0,path_to_image1.jpg,1,0,0,0,1,0,1,0,0,0,0
1,path_to_image2.jpg,0,1,1,0,0,1,0,1,0,0,0
2,path_to_image3.jpg,0,1,0,1,0,0,0,1,1,0,0
3,path_to_image4.jpg,1,0,0,0,1,0,1,0,0,1,0
